# Nesting MicroHH in ERA5

This notebook contains documentation and examples for nesting MicroHH with open boundary conditions in ERA5.

Nesting LES-in-LES is documented separately in `TODO.ipynb`.

In [1]:
# Standard library

# Third-party.
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import ls2d
import cartopy.crs as ccrs

# Local library
from microhhpy.spatial import Domain, plot_domains, Vertical_grid_2nd
from microhhpy.openbc import create_initial_fields_from_era5
from microhhpy.thermo import Basestate_moist

TF = np.float32

## ERA5 data

For now, we use (LS)²D to download and read the ERA5 data. This can be simplified later, as we only need a few 3D fields from ERA5, and do not use the large-scale forcings typically required for a doubly periodic LES.

In [ ]:
settings = {
    'start_date'  : datetime(year=2022, month=4, day=1, hour=12),
    'end_date'    : datetime(year=2022, month=4, day=2, hour=16),
    'central_lon' : 4.8,
    'central_lat' : 53,
    'area_size'   : 5,
    'case_name'   : 'slocs_rf',
    'era5_path'   : '/home/scratch1/bart/LS2D_ERA5/',
    'era5_expver' : 1,
    'cdsapirc'    : '/home/bart/.cdsapirc_ads'
    }

era5 = ls2d.Read_era5(settings)
era5.calculate_forcings(n_av=3, method='2nd')

## Spatial projection

To nest LES in ERA5, a transformation is needed from the LES grid (in meters) to geographic coordinates (latitude / longitude in degrees).

The `Domain()` class provides a simple way to define the domain. The spatial transformation is performed using `pyproj`, based on the `proj_str` definition.

In [ ]:
dom0 = Domain(
    xsize=256_000,
    ysize=256_000,
    itot=128,
    jtot=128,
    n_ghost=3,
    lon=4.8,
    lat=53,
    anchor='center',
    proj_str='+proj=utm +zone=31 +ellps=intl +towgs84=-87,-98,-121,0,0,0,0 +units=m +no_defs +type=crs'
    )

plot_domains([dom0], use_projection=True)

Note that each `Domain` instance includes two projections: one with ghost cells and one without. The version with ghost cells is needed for generating the lateral boundary conditions.

The padded projection actually includes `nghost + 1` ghost cells. The extra cells are needed to create divergence free fields in the east- and north-most ghost cells.

In [ ]:
print(dom0.proj.itot, dom0.proj_pad.itot)

Coordinate pairs are available for each grid point location `(u, v, scalar)`:

In [ ]:
plt.figure()

plt.scatter(dom0.proj.lon,   dom0.proj.lat,   marker='o', facecolor='none', edgecolor='C0', label='scalar')
plt.scatter(dom0.proj.lon_u, dom0.proj.lat_u, marker='>', facecolor='none', edgecolor='C1', label='u')
plt.scatter(dom0.proj.lon_v, dom0.proj.lat_v, marker='^', facecolor='none', edgecolor='C2', label='v')

plt.xlim(dom0.proj.central_lon-0.03, dom0.proj.central_lon+0.03)
plt.ylim(dom0.proj.central_lat-0.03, dom0.proj.central_lat+0.03)

plt.legend()

## Vertical grid

To ensure that the initial LES fields are divergence-free, we need the _exact_ vertical grid definition from MicroHH.

In [6]:
zsize = 3200
ktot = 128
dz = zsize / ktot
z = np.arange(dz/2, zsize, dz)

vgrid = Vertical_grid_2nd(z, zsize, remove_ghost=True)

## Basestate density

Interpolate mean ERA5 fields to LES grid as basis for the LES basestate density. Again, this has to _perfectly_ match the basestate density used by MicroHH.

In [ ]:
les_input = era5.get_les_input(vgrid.z)

bs = Basestate_moist(
    les_input['thl'][0,:].values,
    les_input['qt'][0,:].values,
    float(les_input['ps'][0].values),
    vgrid.z,
    vgrid.zsize,
    remove_ghost=True,
    dtype=TF)

plt.figure()
plt.plot(bs.rho, vgrid.z)
plt.xlabel(r'$\rho$ (kg m$^{-3}$)')
plt.ylabel(r'$z$ (m)')

## Initial conditions

The initial conditions are (not so...) simply the 3D ERA5 fields tri-linearly interpolated to the LES grid.

To reduce the blocky structures that result from interpolating coarse ERA5 data, a Gaussian filter with standard deviation `sigma_h` is applied after interpolation.

### Momentum & divergence
The momentum fields require special treatment because they must be divergence-free. To achieve this, the `(u, v)` velocity components are corrected such that:

1. The resulting fields are divergence-free, and:
2. The domain-mean vertical velocity in LES matches that from ERA5.

Note that we correct the horizontal velocities as `u,v >> w`, and correcting `w` can cause a significant mismatch in subsidence between LES and ERA5.

### Output

The initial conditions are written directly as binary input files (e.g. `u.0000000`) for MicroHH in the specified `output_dir`. By setting a `name_suffix`, the files can optionally be written with a custom name, such as `u_some_name.0000000`. This allows you to overwrite the homogeneous 3D restart files generated during the `init` phase of MicroHH with the fields derived from ERA5.

In [ ]:
fields_era = {
    'u': era5.u[0,:,:,:],
    'v': era5.v[0,:,:,:],
    'w': era5.w[0,:,:,:],
    'thl': era5.thl[0,:,:,:],
    'qt': era5.qt[0,:,:,:],
}

z_era = era5.z[0,:,:,:]

sigma_h = 60_000
correct_div_h = True

u,v,wls = create_initial_fields_from_era5(
    fields_era,
    era5.lons,
    era5.lats,
    z_era,
    vgrid.z,
    vgrid.zh,
    bs.rho,
    bs.rhoh,
    dom0,
    correct_div_h,
    sigma_h,
    output_dir='.',
    name_suffix='era',
    dtype=TF)

In [37]:
u_old = u.copy()
v_old = v.copy()

rho = bs.rho
rhoh = bs.rhoh
npad = 4
dzi = vgrid.dzi
xsize = dom0.xsize
ysize = dom0.ysize
x = dom0.proj_pad.x
y = dom0.proj_pad.y

# Slice of domain without ghost cells.
inner = np.s_[:,npad:-npad, npad:-npad]

# Take mean over interpolated field without ghost cells, to get target mean subsidence velocity.
w_target = wls[inner].mean(axis=(1,2))

# Calculate target horizontal divergence `rho * (du/dx + dv/dy)`.
rho_w = rhoh * w_target
div_uv_target = -(rho_w[1:] - rho_w[:-1]) * dzi[:]

# Calculate actual divergence from interpolated `u,v` fields.
div_u = rho * (u[:, npad:-npad, -npad].mean(axis=1) - u[:, npad:-npad,  npad].mean(axis=1)) / xsize
div_v = rho * (v[:, -npad, npad:-npad].mean(axis=1) - v[:, npad,  npad:-npad].mean(axis=1)) / ysize
div_uv_actual = div_u + div_v

# Required change in horizontal divergence.
diff_div = div_uv_target - div_uv_actual

# Distribute over `u,v`: how to best do this? For now 50/50.
du_dx = diff_div / 2. / rho
dv_dy = diff_div / 2. / rho

# Distance from domain center in `x,y`.
xp = x - xsize / 2.
yp = y - ysize / 2.

# Correct velocities.
u[:,:,:] += du_dx[:,None,None] * xp[None,None,:]
v[:,:,:] += dv_dy[:,None,None] * yp[None,:,None]

In [ ]:
plt.figure()

plt.subplot(231)
plt.title('Uncorrected')
plt.pcolormesh(x, y, u_old[0,:,:])
plt.colorbar()

plt.subplot(232)
plt.title('Corrected')
plt.pcolormesh(x, y, u[0,:,:])
plt.colorbar()

plt.subplot(233)
plt.title('Diff')
plt.pcolormesh(x, y, u[0,:,:]-u_old[0,:,:])
plt.colorbar()

plt.subplot(234)
plt.title('Uncorrected')
plt.pcolormesh(x, y, v_old[0,:,:])
plt.colorbar()

plt.subplot(235)
plt.title('Corrected')
plt.pcolormesh(x, y, v[0,:,:])
plt.colorbar()

plt.subplot(236)
plt.title('Diff')
plt.pcolormesh(x, y, v[0,:,:]-v_old[0,:,:])
plt.colorbar()

In [10]:
def correct_uv_given_w(
        u, v, wls, rho, rhoh, dzi, x, y, xsize, ysize, npad):
    """
    Correct `u,v` fields to ensure that `calc_w_from_uv` results in the same
    mean vertical velocity as the input `wls`.
    """

    # Target mean velocity (domain mean `wls` from ERA5).
    w_target = wls.mean(axis=(1,2))

    # Calculate target horizontal divergence `rho * (du/dx + dv/dy)`.
    rho_w = rhoh * w_target
    div_uv_target = -(rho_w[1:] - rho_w[:-1]) * dzi[:]

    # Calculate actual divergence from interpolated `u,v` fields.
    div_u = rho * (u[:, npad:-npad, -npad].mean(axis=1) - u[:, npad:-npad,  npad].mean(axis=1)) / xsize
    div_v = rho * (v[:, -npad, npad:-npad].mean(axis=1) - v[:, npad,  npad:-npad].mean(axis=1)) / ysize
    div_uv_actual = div_u + div_v

    # Required change in horizontal divergence.
    diff_div = div_uv_target - div_uv_actual

    # Distribute over `u,v`: how to best do this? For now 50/50.
    du_dx = diff_div / 2. / rho
    dv_dy = diff_div / 2. / rho

    # Distance from domain center in `x,y`.
    xp = x - xsize / 2.
    yp = y - ysize / 2.

    # Correct velocities.
    u[:,:,:] += du_dx[:,None,None] * xp[None,None,:]
    v[:,:,:] += dv_dy[:,None,None] * yp[None,:,None]

In [ ]:
thl = np.fromfile('thl_era.0000000', dtype=TF).reshape((vgrid.kmax, dom0.jtot, dom0.itot))

plot_domains([dom0], use_projection=True)
plt.pcolormesh(dom0.proj.lon, dom0.proj.lat, thl[0,:,:], transform=ccrs.PlateCarree())
plt.colorbar()